In [ ]:
!pip install git+https://github.com/pygae/clifford.git@master

In [ ]:
!pip install tensorflow_graphics

In [ ]:
from scipy.spatial.transform import Rotation as R
import numpy as np
from clifford.g3c import *
from clifford.tools.g3c import *
from clifford.tools.g3c.rotor_parameterisation import *
from tensorflow.keras.layers import Input, Dense, LeakyReLU
from keras import backend as K
from sklearn.model_selection import train_test_split
from math import acos
import tensorflow as tf
from matplotlib import pyplot as plt
import keras
from math import e, pi
import tensorflow_graphics as tfg
import tensorflow_graphics.geometry.transformation as tfg_transformation

tot = int(1e5)
'''
#GENERATING THE RANDOM DATASET
i = 0

rot = []
b = []


for i in range(0,tot):

    Q = R.random(random_state = i).as_quat()
    Rot = Q[0] + Q[2]*e13 +Q[1]*e23 + Q[3]*e12

    #rot = np.append(rot, [Q[0], Q[2], Q[1], Q[3]] )
    #B = -2*general_logarithm(Rot)
    B = (1 - Rot)/(1+Rot)
    b = np.append(b, [B[6], B[7], B[10]])


#np.save('Bivectors_cayley.npy', b)
#np.save('Rotors.npy', rot)
'''
rot = np.load('Rotors.npy')
b = np.load('Bivectors_cayley.npy')

rot = np.reshape(rot, [tot, 4])   
b = np.reshape(b, [tot, 3])

#Train - Test Split
r_train, r_test = train_test_split(rot, test_size=0.33, shuffle=False)
b_train, b_test = train_test_split(b, test_size=0.33, shuffle=False)


TRAIN = b_train
TEST = b_test
out_size = 3

nb_epoch = 100
batch_size = 64


r_input = Input(shape=(4))
x = Dense(128)(r_input)
x = LeakyReLU(alpha=0.2)(x)
x = Dense(128)(x)
x = LeakyReLU(alpha=0.2)(x)
x = Dense(128)(x)
x = LeakyReLU(alpha=0.2)(x)
out = Dense(out_size)(x)
print(out)


model = keras.Model(r_input,  out)
model.summary()
model.compile(loss='mae', optimizer="adam")
#es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model_train = model.fit(x = r_train, y = TRAIN, 
                        validation_split=0.3,
                        epochs=nb_epoch,
                        verbose=1,
                        batch_size=batch_size)


loss = model_train.history['loss']
val_loss = model_train.history['val_loss']
epochs = range(nb_epoch)
plt.figure()
plt.plot(epochs, loss, 'b-', label='Training loss')
plt.plot(epochs, val_loss, 'r-', label='Validation loss')
plt.title('Training loss')
plt.legend()
plt.show()

predicted = model.predict(r_test)

M = []
Langle = []

for i in range(0,int(tot*0.33)):
    B = predicted[i][0]*e12 +predicted[i][1]*e13 + predicted[i][2]*e23
    #Rot_pred = e**(-B/2)

    Rot_pred = (1-B)/(1+B)       
    
    #print(Rot_pred)
    B = TEST[i][0]*e12 +TEST[i][1]*e13 + TEST[i][2]*e23
    #Rot_real = e**(-B/2)
    Rot_real = (1-B)/(1+B) 

    #print(Rot_real)

    #print(Rot_real*~Rot_pred)
    cosine = (Rot_real*~Rot_pred)[0]
    #print(cosine)
    #print('-----------------')
    if cosine > 1:
        cosine = 1
    Langle = np.append(Langle, acos(cosine))
    if (acos(cosine)*180/pi) > 90:
        print(acos(cosine)*180/pi)


print(np.max(Langle)*180/pi)
print(np.average(Langle)*180/pi)
print(np.std(Langle)*180/pi)

In [ ]:
M = []
Langle = []

for i in range(0,2):
    B = predicted[i][0]*e12 +predicted[i][1]*e13 + predicted[i][2]*e23

    Rot_pred = (1-B)/(1+B) 
    B = TEST[i][0]*e12 +TEST[i][1]*e13 + TEST[i][2]*e23
    Rot_real = (1-B)/(1+B) 

    print(Rot_pred)
    #print(r_test[i])
    print(Rot_real)

    #print(Rot_real)
    #print('----')

    #R = Rot_real*~Rot_pred

    #print(Rot_real*~Rot_pred)
    cosine = (Rot_real*~Rot_pred)[0]
    #print(cosine)
    #print('-----------------')
    if cosine > 1:
        cosine = 1
    Langle = np.append(Langle, acos(cosine))


print(np.max(Langle)*180/pi)
print(np.average(Langle)*180/pi)
print(np.std(Langle)*180/pi)

In [ ]:
Q = R.random(random_state = i).as_quat()
Rot = Q[0] + Q[2]*e13 +Q[1]*e23 + Q[3]*e12
Rot

B = (1 - Rot)/(1+Rot)
print(B)
print(B[6])
print(B[7])
print(B[10])